In [23]:
import pandas as pd
import numpy as np
import ast
from tabulate import tabulate
import json
import yaml
import matplotlib.pyplot as plt
import itertools
import dataframe_image as dfi


pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [24]:
filename = "REALRUN"  ## insert the name of the file used in the runner.py
df = pd.read_csv(rf"../output/{filename}.csv")

In [25]:
# List of metric names
metric_names = [
    "lift",
    "f1",
    "mse",
    "accuracy",
    "recall",
    "precision",
    "roc_auc",
    "area_under_pr",
    "r2_score",
    "rmse",
]
dataset = df.copy()
# Create columns based on metric names
# Convert string representations to dictionaries
dataset["output_metrics"] = dataset["output_metrics"].apply(ast.literal_eval)
dataset["run_time"] = dataset["run_time"].apply(lambda x: np.round(x / 60, 2))
# Create columns based on metric names
for metric in metric_names:
    dataset[metric] = dataset["output_metrics"].apply(
        lambda x: x.get(metric) if isinstance(x, dict) else np.nan
    )
    dataset[f"{metric}_std"] = dataset[metric].apply(
        lambda x: np.std(x) if isinstance(x, list) else np.nan
    )
    dataset[metric] = dataset[metric].apply(
        lambda x: np.average(x) if isinstance(x, list) else np.nan
    )

base_cols = [
    "dataset",
    "model",
    "run_time",
    "eval_metric",
    "best_score",
    "score_std",
    "output_metrics",
]

base_cols += [i for i in metric_names]
base_cols += [i + "_std" for i in metric_names]

dataset = dataset[base_cols]

In [26]:
tomax = {
    "mse": False,
    "rmse": False,
    "accuracy": True,
    "recall": True,
    "precision": True,
    "roc_auc": True,
    "area_under_pr": True,
    "lift": True,
    "f1": True,
    "r2_score": True,
}


# Define a function to select the first row based on whether to maximize or minimize the "best_score"
def select_first_row(group):
    metric = group["eval_metric"].iloc[0]
    ascending = not tomax.get(
        metric, True
    )  # If metric not in tomax, assume True (maximize)
    return group.sort_values(by="best_score", ascending=ascending).iloc[0]


# Apply the function to each group
filtered_df = dataset.groupby(["dataset", "model", "eval_metric"]).apply(
    select_first_row
)
# Reset the index to get a new DataFrame
filtered_df = filtered_df.reset_index(drop=True)
# Find the rows that maximize the specified metric for each dataset


# Sort the DataFrame based on whether the metric is to be maximized or not
filtered_df["ascending"] = filtered_df["eval_metric"].map(
    {k: not v for k, v in tomax.items()}
)  # Create a new column for ascending order
dfmax = filtered_df.loc[filtered_df["ascending"] == False].sort_values(
    by=["dataset", "eval_metric", "best_score"], ascending=[False, False, False]
)
dfmin = filtered_df.loc[filtered_df["ascending"] == True].sort_values(
    by=["dataset", "eval_metric", "best_score"], ascending=[False, False, True]
)

best_df = pd.concat([dfmax, dfmin])
best_df.drop(columns=["ascending"], inplace=True)

best_df.head(1)

,dataset,model,run_time,eval_metric,best_score,score_std,output_metrics,lift,f1,mse,accuracy,recall,precision,roc_auc,area_under_pr,r2_score,rmse,lift_std,f1_std,mse_std,accuracy_std,recall_std,precision_std,roc_auc_std,area_under_pr_std,r2_score_std,rmse_std
14,titanic,xgb,17.53,roc_auc,0.885801,0.027473,"{'recall': [0.7536231884057971, 0.720588235294...",2.48291,0.770295,NaN,0.836106,0.719096,0.830398,0.885801,0.864007,NaN,NaN,0.119197,0.044381,NaN,0.028001,0.052218,0.038224,0.027473,0.028585,NaN,NaN


# Check State of Run

In [27]:
with open("../configuration/experiment_config.yml", "r") as f:
    config = yaml.safe_load(f)

# Extract the necessary information from the configuration file
included_models = [i.lower() for i in config["include_models"]]
included_datasets = [i.lower() for i in config["include_datasets"]]

# Get all combinations of items from the two lists
combinations_possible = list(itertools.product(included_datasets, included_models))


# Get unique combinations based on 'Column1' and 'Column2'
existing_combinations = df[["dataset", "model"]].drop_duplicates()
# Convert the DataFrame to a list of tuples
existing_combinations = [tuple(row) for row in existing_combinations.values]

missing_combos = [i for i in combinations_possible if i not in existing_combinations]
for i in missing_combos:
    print(f"Missing Combination {i}")

Missing Combination ('iris', 'catboost')
Missing Combination ('iris', 'mlp')
Missing Combination ('iris', 'tabnet')
Missing Combination ('iris', 'gate')
Missing Combination ('iris', 'resnet')
Missing Combination ('iris', 's1dcnn')
Missing Combination ('iris', 'categoryembedding')
Missing Combination ('iris', 'fttransformer')
Missing Combination ('iris', 'tabtransformer')
Missing Combination ('iris', 'gandalf')
Missing Combination ('iris', 'autoint')
Missing Combination ('titanic', 'catboost')
Missing Combination ('titanic', 'mlp')
Missing Combination ('titanic', 'tabnet')
Missing Combination ('titanic', 'gate')
Missing Combination ('titanic', 'resnet')
Missing Combination ('titanic', 's1dcnn')
Missing Combination ('titanic', 'categoryembedding')
Missing Combination ('titanic', 'fttransformer')
Missing Combination ('titanic', 'tabtransformer')
Missing Combination ('titanic', 'gandalf')
Missing Combination ('titanic', 'autoint')
Missing Combination ('breastcancer', 'catboost')
Missing Co

In [28]:
[i for i in missing_combos if "xgb" in i]

[('covertype', 'xgb')]

In [29]:
[i for i in missing_combos if "node" in i]

[('adult', 'node'),
 ('housing', 'node'),
 ('creditcard', 'node'),
 ('covertype', 'node')]

In [9]:
best_df["dataset"].value_counts()

dataset
titanic          2
iris             2
heloc            2
diabetes         2
breastcancer     2
ageconditions    2
housing          1
creditcard       1
adult            1
Name: count, dtype: int64

# Execution Time Visualization

In [10]:
# Group by the "dataset" column and aggregate "run_time" using the sum function
hyperopt_evals = 100
num_parallel = 3
efficiency_estimate = 0.8
aggregated_df = best_df.groupby("model")["run_time"].sum().reset_index()
# Rename the aggregated column for clarity
aggregated_df = aggregated_df.rename(columns={"run_time": "sum_run_time"})
aggregated_df["total_search_hours_estimate"] = (
    aggregated_df["sum_run_time"] * 100 / 60 / 3 * 0.8
)
aggregated_df.sort_values("total_search_hours_estimate", ascending=False).head(20)

,model,sum_run_time,total_search_hours_estimate
1,xgb,473.41,210.404444
0,node,426.54,189.573333


In [12]:
# Convert DataFrame to ASCII table
class Format:
    end = "\033[0m"
    underline = "\033[4m"


def make_results_table(
    df, dataset_name, display_cols, metric_cols, image_name="", dpi=1200, image_path="/home/boom/sdev/WTabRun/notebooks/tables/",
    image_folder = ""
):
    result_df = df.loc[df["dataset"] == dataset_name].reset_index(drop=True).copy()

    # Create a dictionary to store the indices of the rows with the highest values for each metric column
    max_indices = {}
    for metric in metric_cols:
        max_indices[metric] = result_df[metric].idxmax()

    # Modify all columns with std to include relative std
    result_df["best_score"] = result_df.apply(
        lambda row: f"{row['best_score']:.4f} ± ({row['score_std']:.4f})", axis=1
    )
    for metric in metric_cols:
        result_df[metric] = result_df.apply(
            lambda row: f"{row[metric]:.4f} ± ({row[metric+'_std']:.4f})", axis=1
        )
        # Drop the corresponding std column
        result_df.drop(columns=[metric + "_std"], inplace=True)

    result_df[display_cols].to_csv(rf"{image_path}/{image_folder}/{image_name}.csv")

    return result_df[display_cols]


# Define a custom styling function
def highlight_max_row(s):
    is_max = s == s.max()
    return ["background-color: green" if v else "" for v in is_max]


def highlight_min_row(s):
    is_max = s == s.min()
    return ["background-color: green" if v else "" for v in is_max]

In [13]:
folder = "hyperopt"

# Titanic


In [14]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "titanic"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png" , dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,xgb,0.8361 ± (0.0280),0.8858 ± (0.0275),2.4829 ± (0.1192),0.7703 ± (0.0444),0.7191 ± (0.0522),0.8304 ± (0.0382),0.8640 ± (0.0286)
1,node,0.8216 ± (0.0262),0.8817 ± (0.0219),2.5442 ± (0.0816),0.7704 ± (0.0349),0.7806 ± (0.0460),0.7626 ± (0.0452),0.8550 ± (0.0207)


# Housing

In [15]:
display_cols = ["model", "mse", "r2_score", "rmse"]
metric_cols = ["mse", "r2_score", "rmse"]
dataset_name = "housing"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=["r2_score"], axis=0).apply(
    highlight_min_row, subset=["mse", "rmse"], axis=0
)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,mse,r2_score,rmse
0,xgb,0.1870 ± (0.0080),0.8595 ± (0.0069),0.4323 ± (0.0093)


# Heloc

In [16]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "heloc"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,node,0.7183 ± (0.0049),0.7977 ± (0.0042),1.7326 ± (0.0347),0.7224 ± (0.0079),0.7025 ± (0.0166),0.7437 ± (0.0049),0.7968 ± (0.0068)
1,xgb,0.7186 ± (0.0047),0.7929 ± (0.0056),1.7197 ± (0.0508),0.7411 ± (0.0040),0.7716 ± (0.0115),0.7131 ± (0.0078),0.7905 ± (0.0093)


# Diabetes

In [17]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "diabetes"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,xgb,0.8888 ± (0.0002),0.6817 ± (0.0055),2.4958 ± (0.0868),0.0326 ± (0.0095),0.0168 ± (0.0051),0.5628 ± (0.0141),0.2341 ± (0.0081)
1,node,0.6364 ± (0.0280),0.6779 ± (0.0060),2.4271 ± (0.0728),0.2726 ± (0.0069),0.6097 ± (0.0357),0.1759 ± (0.0070),0.2199 ± (0.0090)


# Creditcard

In [18]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "creditcard"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,xgb,0.9996 ± (0.0001),0.9828 ± (0.0082),9.6547 ± (0.2193),0.8584 ± (0.0211),0.7886 ± (0.0216),0.9420 ± (0.0262),0.8548 ± (0.0246)


# Adult

In [19]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = ["accuracy", "roc_auc", "f1", "recall", "precision", "area_under_pr"]
dataset_name = "adult"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,f1,recall,precision,area_under_pr
0,xgb,0.8737 ± (0.0034),0.9286 ± (0.0013),0.7130 ± (0.0079),0.6516 ± (0.0100),0.7873 ± (0.0100),0.8296 ± (0.0037)


# Iris

In [20]:
display_cols = ["model", "accuracy", "f1"]
metric_cols = [
    "accuracy",
    "f1",
]
dataset_name = "iris"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,f1
0,node,0.9800 ± (0.0267),0.9800 ± (0.0267)
1,xgb,0.9600 ± (0.0389),0.9598 ± (0.0390)


# Covertype

In [21]:
display_cols = ["model", "accuracy", "f1"]
metric_cols = ["accuracy", "f1"]
dataset_name = "covertype"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

ValueError: attempt to get argmax of an empty sequence

# Breastcancer

In [22]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "breastcancer"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.sort_values("f1", ascending=False).style.apply(
    highlight_max_row, subset=metric_cols, axis=0
)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,node,0.9859 ± (0.0119),0.9943 ± (0.0063),1.5911 ± (0.0121),0.9889 ± (0.0093),0.9944 ± (0.0068),0.9836 ± (0.0158),0.9959 ± (0.0049)
1,xgb,0.9631 ± (0.0129),0.9909 ± (0.0066),1.5939 ± (0.0100),0.9708 ± (0.0099),0.9748 ± (0.0106),0.9673 ± (0.0235),0.9939 ± (0.0046)


In [88]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "ageconditions"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.sort_values("f1", ascending=False).style.apply(
    highlight_max_row, subset=metric_cols, axis=0
)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
2,s1dcnn,0.9473 ± (0.0106),0.9606 ± (0.0134),5.0003 ± (0.5966),0.8528 ± (0.0292),0.8705 ± (0.0372),0.8369 ± (0.0378),0.8348 ± (0.0594)
0,catboost,0.9416 ± (0.0209),0.9670 ± (0.0245),5.2348 ± (0.4964),0.8162 ± (0.0640),0.7407 ± (0.0630),0.9140 ± (0.0912),0.8886 ± (0.0808)
1,xgb,0.9157 ± (0.0217),0.9572 ± (0.0320),5.2348 ± (0.2364),0.7329 ± (0.0561),0.6571 ± (0.0641),0.8468 ± (0.1150),0.8776 ± (0.0628)
3,gandalf,0.8850 ± (0.0543),0.9474 ± (0.0372),4.7674 ± (0.5775),0.7283 ± (0.0981),0.8429 ± (0.0857),0.6537 ± (0.1291),0.8256 ± (0.0783)
5,gate,0.8688 ± (0.0400),0.9120 ± (0.0325),4.6609 ± (0.5058),0.6835 ± (0.0546),0.7870 ± (0.0224),0.6106 ± (0.0829),0.7312 ± (0.0775)
4,autoint,0.8785 ± (0.0177),0.8626 ± (0.0717),4.6617 ± (0.4189),0.6377 ± (0.1378),0.6879 ± (0.2380),0.7070 ± (0.1571),0.7265 ± (0.0928)
7,tabtransformer,0.7858 ± (0.1616),0.8201 ± (0.1605),3.8925 ± (1.2428),0.5907 ± (0.1706),0.7403 ± (0.1392),0.5132 ± (0.1821),0.6357 ± (0.2167)
8,categoryembedding,0.8509 ± (0.0420),0.7861 ± (0.2041),3.8852 ± (1.4910),0.5458 ± (0.2425),0.6074 ± (0.2805),0.5261 ± (0.1969),0.6478 ± (0.2551)
6,mlp,0.8734 ± (0.0513),0.8645 ± (0.0338),3.9998 ± (0.9324),0.5349 ± (0.2398),0.4623 ± (0.2329),0.6529 ± (0.2183),0.6482 ± (0.1373)
11,tabnet,0.3976 ± (0.2265),0.5749 ± (0.0990),1.5213 ± (0.6885),0.2953 ± (0.0397),0.7069 ± (0.2852),0.2241 ± (0.1032),0.2591 ± (0.0739)
